In [64]:
import json
import numpy as np
import pandas as pd
import torch.nn as nn
import torch
import collections
import math
import gzip
import copy
from tqdm import tqdm
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import StepLR
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
from google.colab import files
from google.colab import drive
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device_2 = "cpu"
path = '/content/drive'
drive.mount(path)
train_path=  "/content/drive/My Drive/CSCI544/HW4/hw4/data/train"
dev_path=  "/content/drive/My Drive/CSCI544/HW4/hw4/data/dev"
test_path=  "/content/drive/My Drive/CSCI544/HW4/hw4/data/test"
glove_path=  "/content/drive/My Drive/CSCI544/HW4/hw4/glove.6B.100d.gz"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [82]:
with open(train_path) as f:
  train = f.read()
  train_set = train.split('\n\n')
train_set = [t.split('\n') for t in train_set]
train_set[-1].pop(-1)


with open(dev_path) as f:
  dev = f.read()
  dev_set = dev.split('\n\n')
dev_set = [t.split('\n') for t in dev_set]
dev_set[-1].pop(-1)

with open(test_path) as f:
  test = f.read()
  test_set = test.split('\n\n')
test_set = [t.split('\n') for t in test_set]
test_set[-1].pop(-1)

''

In [83]:
#Generating input data frame:

#word index and tag index
train_word_set = []
for i in range(len(train_set)-1):
  train_word_set +=([e.split()[1] for e in train_set[i]])

train_word_set += [train_set[-1][0].split()[1]]


train_tag_set = []
for i in range(len(train_set)-1):
  train_tag_set +=([e.split()[2] for e in train_set[i]])
train_tag_set += [train_set[-1][0].split()[2]]
train_tag_set.append('<PAD>')


train_word_set = list(set(train_word_set))
train_tag_set = list(set(train_tag_set))
train_word_set.append('<UNK>')
train_word_set.append('<PAD>')

num_tags = len(train_tag_set)
num_words = len(train_word_set)

idx2word_train = {index:word for  index, word in enumerate(train_word_set)}
word2idx_train = {word:index for  index, word in enumerate(train_word_set)}


idx2tag_train = {index:tag for  index, tag in enumerate(train_tag_set)}
tag2idx_train = {tag:index for  index, tag in enumerate(train_tag_set)}


#train
for sentence in train_set:
  for i in range(len(sentence)):
    temp1 = sentence[i].split()[1]
    temp2 = sentence[i].split()[2]
    sentence[i] +=' '
    sentence[i] +=str(word2idx_train[temp1])
    sentence[i] +=' '
    sentence[i] +=str(tag2idx_train[temp2])
    
train_set_reformatted = []
for sentence in train_set:
  row = []
  s = [x.split()[3] for x in sentence]
  t = [y.split()[4] for y in sentence]
  row.append(s)
  row.append(t)
  train_set_reformatted.append(row)


df_train = pd.DataFrame(train_set_reformatted,columns=['sentence','tag'])
vocab_size = len(train_word_set)
sentences_train = df_train["sentence"].tolist()
tags_train = df_train['tag'].tolist()
sentences_train = [[int(num) for num in lst] for lst in sentences_train]
tags_train = [[int(num) for num in lst] for lst in tags_train]


#dev
for sentence in dev_set:
  for i in range(len(sentence)):
    temp1 = sentence[i].split()[1]
    temp2 = sentence[i].split()[2]
    sentence[i] +=' '
    if temp1 not in word2idx_train:
      sentence[i] +=str(word2idx_train['<UNK>'])

    else:
      sentence[i] +=str(word2idx_train[temp1])

    sentence[i] +=' '

    sentence[i] +=str(tag2idx_train[temp2])



for sentence in test_set:
  for i in range(len(sentence)):
    temp1 = sentence[i].split()[1]
    #temp2 = sentence[i].split()[2]
    sentence[i] +=' '
    if temp1 not in word2idx_train:
      sentence[i] +=str(word2idx_train['<UNK>'])

    else:
      sentence[i] +=str(word2idx_train[temp1])

dev_set_reformatted = []
for sentence in dev_set:
  row = []
  s = [x.split()[3] for x in sentence]
  t = [y.split()[4] for y in sentence]
  row.append(s)
  row.append(t)
  dev_set_reformatted.append(row)


df_dev = pd.DataFrame(dev_set_reformatted,columns=['sentence','tag'])
sentences_dev = df_dev["sentence"].tolist()
tags_dev = df_dev['tag'].tolist()
sentences_dev = [[int(num) for num in lst] for lst in sentences_dev]
tags_dev = [[int(num) for num in lst] for lst in tags_dev]

#test
test_set_reformatted = []
for sentence in test_set:
  row = []
  s = [x.split()[2] for x in sentence]
  #t = [y.split()[4] for y in sentence]
  #row.append(s)
  #row.append(t)
  test_set_reformatted.append(s)


df_test = pd.DataFrame({'sentence': test_set_reformatted})
sentences_test = df_test["sentence"].tolist()
sentences_test = [[int(num) for num in lst] for lst in sentences_test]



padded_sentences_train = pad_sequence([torch.tensor(s) for s in sentences_train], batch_first=True,padding_value=word2idx_train['<PAD>'])
padded_tags_train = pad_sequence([torch.tensor(t) for t in tags_train], batch_first=True, padding_value=tag2idx_train['<PAD>'])
padded_sentences_dev = pad_sequence([torch.tensor(s) for s in sentences_dev], batch_first=True,padding_value=word2idx_train['<PAD>'])
padded_tags_dev = pad_sequence([torch.tensor(t) for t in tags_dev], batch_first=True, padding_value=tag2idx_train['<PAD>'])

-------------------------------Task 1-----------------------------

In [84]:
embedding_dim = 100
lstm_layers = 1
lstm_hidden_dim = 256
lstm_dropout = 0.33
linear_output_dim = 128
learning_rate = 0.2
num_epochs = 150
batch_size = 32
alpha = 5

class BLSTM(nn.Module):
    def __init__(self, embedding_dim, lstm_layers, lstm_hidden_dim, lstm_dropout, linear_output_dim,alpha):
        super(BLSTM, self).__init__()
        self.embedding = nn.Embedding(num_embeddings=num_words, embedding_dim=embedding_dim)
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=lstm_hidden_dim, num_layers=lstm_layers, dropout=lstm_dropout, bidirectional=True,batch_first=True)
        self.linear = nn.Linear(in_features=lstm_hidden_dim*2, out_features=linear_output_dim)
        self.activation = nn.ELU(alpha= alpha)
        self.dropout = nn.Dropout(lstm_dropout)
        self.classifier = nn.Linear(in_features=linear_output_dim, out_features=num_tags)

    def forward(self, x):
        embedded = self.embedding(x)
        lstm_out, _ = self.lstm(embedded)
        linear_out = self.dropout(lstm_out)
        linear_out = self.linear(linear_out)
      
        activation_out = self.activation(linear_out)
        classifier_out = self.classifier(activation_out)
        return classifier_out

class NERDataset(Dataset):
    def __init__(self, sentences, tags):
        self.sentences = sentences
        self.tags = tags

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, index):
        sentence = self.sentences[index]
        tag = self.tags[index]
        
        sentence_tensor = torch.tensor(sentence, dtype=torch.long)
        tag_tensor = torch.tensor(tag, dtype=torch.long)

        return sentence_tensor, tag_tensor

In [ ]:
train_dataset = NERDataset(padded_sentences_train,padded_tags_train)
val_dataset = NERDataset(padded_sentences_dev,padded_tags_dev)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

model = BLSTM(embedding_dim, lstm_layers, lstm_hidden_dim, lstm_dropout, linear_output_dim,alpha)
criterion = nn.CrossEntropyLoss(ignore_index=tag2idx_train['<PAD>'])
optimizer = optim.SGD(model.parameters(), lr=learning_rate)
scheduler = StepLR(optimizer, step_size=10, gamma=0.8)
model.to(device)
#model.to(device_2)

In [ ]:
for epoch in range(num_epochs):
    train_loss = 0.0
    model.train()
    for sentences, tags in tqdm(train_loader):
        optimizer.zero_grad()
        sentences = sentences.to(device)
        tags = tags.to(device)
        outputs = model(sentences)
        #mask = (tags != tag2idx_train['<PAD>']).float()
        loss = criterion(outputs.view(-1, num_tags), tags.view(-1))
        #loss = (loss * mask.view(-1)).sum() /mask.sum()
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
    scheduler.step()

    
    print('Epoch [{}/{}], Train Loss: {:.4f}'.format(epoch+1, num_epochs, train_loss/len(train_loader)))


In [87]:
#generate test result
print('Epoch [{}/{}], Train Loss: {:.4f}'.format(epoch+1, num_epochs, train_loss/len(train_loader)))
test_pred_result = []
model.eval()
for s in sentences_test:
  test_tensor = torch.tensor(s).to(device)

  with torch.no_grad():
    outputs = model(test_tensor)
    outputs = outputs.cpu().numpy()
    predicted_tags = np.argmax(outputs, axis=1)
    test_pred_result.append(list(predicted_tags)) 


with open('test_prediction.txt', 'w') as f:
  for i in range(df_test.shape[0]):
    s_pred = test_pred_result[i]
    
    l = len(s_pred)
    ss = sentences_test[i]
    for j in range(l):
      word =idx2word_train[ss[j]]
      f.write(f'{j+1} {word} {idx2tag_train[s_pred[j]]}\n')

    if i!=df_test.shape[0]-1:
      f.write('\n')


with open('test1.out', 'w') as f:
  for i in range(df_test.shape[0]):
    s_pred = test_pred_result[i]
    
    l = len(s_pred)
    ss = sentences_test[i]
    for j in range(l):
      word =idx2word_train[ss[j]]
      f.write(f'{j+1} {word} {idx2tag_train[s_pred[j]]}\n')
    if i!=df_test.shape[0]-1:
      f.write('\n')    

#generate dev result
dev_pred_result = []
model.eval()
for s in sentences_dev:
  dev_tensor = torch.tensor(s).to(device)
  with torch.no_grad():
    outputs = model(dev_tensor)
    outputs = outputs.cpu().numpy()
    predicted_tags = np.argmax(outputs, axis=1)
    dev_pred_result.append(list(predicted_tags))
   

with open('dev_prediction.txt', 'w') as f:
  for i in range(df_dev.shape[0]):
    s_pred = dev_pred_result[i]
    s_true = tags_dev[i]
    l = len(s_pred)
    ss = sentences_dev[i]
    for j in range(l):
      word =idx2word_train[ss[j]]
      f.write(f'{j+1} {word} {idx2tag_train[s_true[j]]} {idx2tag_train[s_pred[j]]}\n')
    if i!=df_dev.shape[0]-1:
      f.write('\n')


with open('dev1.out', 'w') as f:
  for i in range(df_dev.shape[0]):
    s_pred = dev_pred_result[i]
    l = len(s_pred)
    ss = sentences_dev[i]
    for j in range(l):
      word =idx2word_train[ss[j]]
      f.write(f'{j+1} {word} {idx2tag_train[s_pred[j]]}\n')
    if i!=df_dev.shape[0]-1:
      f.write('\n')
    

Epoch [150/150], Train Loss: 0.0138


In [88]:
#Save the model and results for task 1
model_state_dict = model.state_dict()
model_path = 'blstm1.pt'
torch.save(model_state_dict, model_path)
files.download('blstm1.pt') 

files.download('test_prediction.txt') 
files.download('dev_prediction.txt') 
files.download('dev1.out')
files.download('test1.out')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

-----------------------Task 2----------------------------

In [89]:
with open(train_path) as f:
  train = f.read()
  train_set = train.split('\n\n')
train_set = [t.split('\n') for t in train_set]
train_set[-1].pop(-1)


with open(dev_path) as f:
  dev = f.read()
  dev_set = dev.split('\n\n')
dev_set = [t.split('\n') for t in dev_set]
dev_set[-1].pop(-1)

with open(test_path) as f:
  test = f.read()
  test_set = test.split('\n\n')
test_set = [t.split('\n') for t in test_set]
test_set[-1].pop(-1)

''

In [90]:
#loading glove
embeddings = []
glove_words=[]

#processing glove
with gzip.open(glove_path, 'rt') as f:
  for line in f:
    line = line.strip()
    parts = line.split()
    word = parts[0]
    glove_words.append(word)
    embedding = np.array([float(p) for p in parts[1:]])
    embeddings.append(embedding)


In [91]:
embeddings = torch.tensor(embeddings)
glove_words = ["<PAD>","<UNK>"] + glove_words
pad_vec = torch.zeros(1, 100)
unk_vec = torch.randn(1, 100)
glove_vectors = torch.cat([pad_vec, unk_vec, embeddings], dim=0)
word2idx_glove = {word: idx for idx, word in enumerate(glove_words)}
idx2word_glove = {idx: word for idx, word in enumerate(glove_words)}
num_words = len(glove_words)


tag2idx_train['<PAD>'],tag2idx_train['B-LOC']=tag2idx_train['B-LOC'],tag2idx_train['<PAD>']
num_tags = len(tag2idx_train)


def is_capital(word):
  return int(word != word.lower()) +1

In [92]:
#train task2

for sentence in train_set:
  for i in range(len(sentence)):
    temp1 = sentence[i].split()[1]
    temp2 = sentence[i].split()[2]
    sentence[i] +=' '
    sentence[i] +=str(word2idx_glove[temp1.lower()]) if temp1.lower() in word2idx_glove else str(word2idx_glove['<UNK>'])
    sentence[i] +=' '
    sentence[i] +=str(tag2idx_train[temp2])
    sentence[i] +=' '
    sentence[i] +=str(is_capital(temp1))
    
train_set_reformatted = []
for sentence in train_set:
  row = []
  s = [x.split()[3] for x in sentence]
  t = [y.split()[4] for y in sentence]
  c = [z.split()[5] for z in sentence]
  row.append(s)
  row.append(t)
  row.append(c)
  train_set_reformatted.append(row)

df_train = pd.DataFrame(train_set_reformatted,columns=['sentence','tag','cap'])
sentences_train = df_train["sentence"].tolist()
tags_train = df_train['tag'].tolist()
caps_train = df_train['cap'].tolist()
sentences_train = [[int(num) for num in lst] for lst in sentences_train]
tags_train = [[int(num) for num in lst] for lst in tags_train]
caps_train = [[int(num) for num in lst] for lst in caps_train]



#dev task2
for sentence in dev_set:
  for i in range(len(sentence)):
    temp1 = sentence[i].split()[1]
    temp2 = sentence[i].split()[2]
    sentence[i] +=' '
    sentence[i] +=str(word2idx_glove[temp1.lower()]) if temp1.lower() in word2idx_glove else str(word2idx_glove['<UNK>'])
    sentence[i] +=' '
    sentence[i] +=str(tag2idx_train[temp2])
    sentence[i] +=' '
    sentence[i] +=str(is_capital(temp1))


dev_set_reformatted = []
for sentence in dev_set:
  row = []
  s = [x.split()[3] for x in sentence]
  t = [y.split()[4] for y in sentence]
  c = [z.split()[5] for z in sentence]
  row.append(s)
  row.append(t)
  row.append(c)
  dev_set_reformatted.append(row)


df_dev = pd.DataFrame(dev_set_reformatted,columns=['sentence','tag','cap'])
sentences_dev = df_dev["sentence"].tolist()
tags_dev = df_dev['tag'].tolist()
caps_dev = df_dev['cap'].tolist()
sentences_dev = [[int(num) for num in lst] for lst in sentences_dev]
tags_dev = [[int(num) for num in lst] for lst in tags_dev]
caps_dev = [[int(num) for num in lst] for lst in caps_dev]


#test task2
for sentence in test_set:
  for i in range(len(sentence)):
    temp1 = sentence[i].split()[1]
    sentence[i] +=' '
    sentence[i] +=str(word2idx_glove[temp1.lower()]) if temp1.lower() in word2idx_glove else str(word2idx_glove['<UNK>'])
    sentence[i] +=' '
    sentence[i] +=str(is_capital(temp1))


test_set_reformatted = []
for sentence in test_set:
  row = []
  s = [x.split()[2] for x in sentence]
  c = [z.split()[3] for z in sentence]
  row.append(s)
  row.append(c)
  test_set_reformatted.append(row)


df_test = pd.DataFrame(test_set_reformatted,columns=['sentence','cap'])
sentences_test = df_test["sentence"].tolist()
caps_test = df_test['cap'].tolist()
sentences_test = [[int(num) for num in lst] for lst in sentences_test]
caps_test = [[int(num) for num in lst] for lst in caps_test]


In [99]:
embedding_dim = 100
lstm_layers = 1
lstm_hidden_dim = 256
lstm_dropout = 0.33
linear_output_dim = 128
learning_rate = 0.2
num_epochs = 50
batch_size = 32
alpha = 5


class BLSTM_glove(nn.Module):
    def __init__(self, embedding_dim, lstm_layers, lstm_hidden_dim, lstm_dropout, linear_output_dim,alpha):
        super(BLSTM_glove, self).__init__()
        
       
        self.embedding = nn.Embedding.from_pretrained(glove_vectors, freeze=True)
        self.capital_embedding = nn.Embedding(3, embedding_dim)
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=lstm_hidden_dim, num_layers=lstm_layers, bidirectional=True,batch_first=True)
        self.linear = nn.Linear(in_features=lstm_hidden_dim*2, out_features=linear_output_dim)
        self.dropout = nn.Dropout(lstm_dropout)
        self.activation = nn.ELU(alpha= alpha)
        self.classifier = nn.Linear(in_features=linear_output_dim, out_features=num_tags)

    def forward(self,x,capital_x):
        
        embedded = self.embedding(x)
        capital_embedded = self.capital_embedding(capital_x)
        capital_embedded= capital_embedded.float()
        embedded = embedded.float()
        lstm_out, _ = self.lstm(embedded+capital_embedded)
        linear_out = self.dropout(lstm_out)
        linear_out = self.linear(linear_out)
        activation_out = self.activation(linear_out)
        classifier_out = self.classifier(activation_out)
        return classifier_out

class NERDataset(Dataset):
    def __init__(self, sentences, tags,caps):
        self.sentences = sentences
        self.tags = tags
        self.caps = caps

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, index):
        sentence = self.sentences[index]
        tag = self.tags[index]
        cap = self.caps[index]
        sentence_tensor = torch.tensor(sentence, dtype=torch.long)
        tag_tensor = torch.tensor(tag,dtype = torch.long)
        cap_tensor = torch.tensor(cap,dtype = torch.long)

        return sentence_tensor,tag_tensor,cap_tensor



def collate_fn(batch):
    padded_sentences = pad_sequence([b[0] for b in batch], batch_first=True, padding_value=word2idx_glove['<PAD>'])
    padded_tags = pad_sequence([b[1] for b in batch], batch_first=True, padding_value=tag2idx_train['<PAD>'])
    padded_caps = pad_sequence([b[2] for b in batch], batch_first=True, padding_value=0)

    return padded_sentences, padded_tags, padded_caps

In [100]:
#train_sentences, val_sentences, train_tags, val_tags = train_test_split(sentences_train, tags_train, test_size=0.2)
train_dataset = NERDataset(sentences_train,tags_train,caps_train)
dev_dataset = NERDataset(sentences_dev,tags_dev,caps_dev)
train_loader = DataLoader(train_dataset, collate_fn=collate_fn, batch_size=32)
val_loader = DataLoader(dev_dataset, collate_fn=collate_fn, batch_size=32)

model_2 = BLSTM_glove(embedding_dim, lstm_layers, lstm_hidden_dim, lstm_dropout, linear_output_dim,alpha)
criterion = nn.CrossEntropyLoss(ignore_index=tag2idx_train['<PAD>'])
optimizer = optim.SGD(model_2.parameters(), lr=learning_rate)
scheduler = StepLR(optimizer, step_size=20, gamma=0.8)
model_2.to(device)
#model.to(device_2)

BLSTM_glove(
  (embedding): Embedding(400002, 100)
  (capital_embedding): Embedding(3, 100)
  (lstm): LSTM(100, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=128, bias=True)
  (dropout): Dropout(p=0.33, inplace=False)
  (activation): ELU(alpha=5)
  (classifier): Linear(in_features=128, out_features=10, bias=True)
)

In [ ]:
for epoch in range(num_epochs):
    train_loss = 0.0
    model_2.train()
    for sentences, tags,capital in tqdm(train_loader):
        optimizer.zero_grad()
        sentences = sentences.to(device)
        tags = tags.to(device)
        capitals = capital.to(device)

        outputs = model_2(sentences, capitals)
        #mask = (tags != tag2idx_train['<PAD>']).float()
        loss = criterion(outputs.view(-1, num_tags), tags.view(-1))
        #loss = (loss * mask.view(-1)).sum() /mask.sum()
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
    scheduler.step()

    
    print('Epoch [{}/{}], Train Loss: {:.4f}'.format(epoch+1, num_epochs, train_loss/len(train_loader)))


In [97]:
#generate test result
print('Epoch [{}/{}], Train Loss: {:.4f}'.format(epoch+1, num_epochs, train_loss/len(train_loader)))
test_pred_result = []
model_2.eval()
for i in range(len(sentences_test)):
  sentence_test_tensor = torch.tensor(sentences_test[i]).to(device)
  
  capital_test_tensor = torch.tensor(caps_test[i]).to(device)

  with torch.no_grad():
    outputs = model_2(sentence_test_tensor,capital_test_tensor)
    outputs = outputs.cpu().numpy()
    predicted_tags = np.argmax(outputs, axis=1)
    test_pred_result.append(list(predicted_tags))
   


with open('test_prediction_2.txt', 'w') as f:
  for i in range(df_test.shape[0]):
    s_pred = test_pred_result[i]
    
    l = len(s_pred)
    ss = sentences_test[i]
    for j in range(l):
      word =idx2word_glove[ss[j]]
      f.write(f'{j+1} {word} {idx2tag_train[s_pred[j]]}\n')
      
    if i!=df_test.shape[0]-1:
      f.write('\n')


with open('test2.out', 'w') as f:
  for i in range(df_test.shape[0]):
    s_pred = test_pred_result[i]
    
    l = len(s_pred)
    ss = sentences_test[i]
    for j in range(l):
      word =idx2word_glove[ss[j]]
      f.write(f'{j+1} {word} {idx2tag_train[s_pred[j]]}\n')
    if i!=df_test.shape[0]-1:
      f.write('\n')    
 


#generate dev result
dev_pred_result = []
model_2.eval()
for i in range(len(sentences_dev)):
  sentence_dev_tensor = torch.tensor(sentences_dev[i]).to(device)
  
  capital_tensor_dev = torch.tensor(caps_dev[i]).to(device)
  with torch.no_grad():
    outputs = model_2(sentence_dev_tensor,capital_tensor_dev)
    outputs = outputs.cpu().numpy()
    predicted_tags = np.argmax(outputs, axis=1)
    dev_pred_result.append(list(predicted_tags))
    

with open('dev_prediction_2.txt', 'w') as f:
  for i in range(df_dev.shape[0]):
    s_pred = dev_pred_result[i]
    s_true = tags_dev[i]
    l = len(s_pred)
    ss = sentences_dev[i]
    for j in range(l):
      word =idx2word_glove[ss[j]]
      f.write(f'{j+1} {word} {idx2tag_train[s_true[j]]} {idx2tag_train[s_pred[j]]}\n')
    if i!=df_dev.shape[0]-1:
      f.write('\n')


with open('dev2.out', 'w') as f:
  for i in range(df_dev.shape[0]):
    s_pred = dev_pred_result[i]
    l = len(s_pred)
    ss = sentences_dev[i]
    for j in range(l):
      word =idx2word_glove[ss[j]]
      f.write(f'{j+1} {word} {idx2tag_train[s_pred[j]]}\n')
    if i!=df_dev.shape[0]-1:
      f.write('\n')
     

Epoch [80/80], Train Loss: 0.0180


In [98]:
#Save the model and results for task 2
model_state_dict_2 = model_2.state_dict()
model_path = 'blstm2.pt'
torch.save(model_state_dict_2, model_path)
files.download('blstm2.pt') 

files.download('test_prediction_2.txt') 
files.download('dev_prediction_2.txt') 
files.download('dev2.out')
files.download('test2.out')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>